In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Conv2D, MaxPooling2D
# from keras.layers import 
from keras import backend as K
import random
import copy

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

Using TensorFlow backend.


In [2]:
batch_size = 2000
num_classes = 10
epochs = 1
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization(momentum=0.75, epsilon=0.001))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
# model.add(BatchNormalization(momentum=0.75, epsilon=0.001))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# model.fit(x_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

W0915 16:50:52.912473 16068 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0915 16:50:52.924475 16068 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0915 16:50:52.926476 16068 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0915 16:50:52.948481 16068 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0915 16:50:52.960484 16068 deprecation_wrapper.py:119] From C:\ProgramData\Ana

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


W0915 16:50:53.163529 16068 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0915 16:50:53.213541 16068 deprecation.py:506] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0915 16:50:53.263552 16068 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Test loss: 2.3082639640808105
Test accuracy: 0.0858


In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 12, 12, 64)        256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
__________

## data[2].astype(dtype=float32)

In [4]:
import numpy as np

for i in range(0,7,2):
    print(str(i+1) + '. ' + str(model.get_weights()[i].shape) + ', ' + str(i+2) + '. '+str(model.get_weights()[i+1].shape))

1. (3, 3, 1, 32), 2. (32,)
3. (3, 3, 32, 64), 4. (64,)
5. (64,), 6. (64,)
7. (64,), 8. (64,)


In [5]:
# gen_normal_noise
def gen_random_tensor(tensor_shape, scaling='normalized'):
    rand_numbers = np.random.normal(size=tensor_shape)
    if scaling == 'normalized':
        rand_numbers = rand_numbers/(np.max(rand_numbers))
    
    return rand_numbers

def gen_normal_noise(weights_shape, scaling='normalized'):
    noise = np.random.normal(size=weights_shape)
    if scaling == 'normalized':
        noise = noise/(np.max(noise))
    else:
        noise = noise/2
    return noise

In [6]:
# def generate_noisy_model(keras_model):
#     len_layers = len(keras_model.layers)
#     new_model = keras_model.clone_model(model, input_tensors = NULL)

#     for i in range(len_layers):
#         len_list_weights = len(keras_model.layers[i].get_weights())
#         weight_list_dummy = []
        
#         for j in range(len_list_weights):
#             if keras_model.layers[i].get_weights()[j].size > 0:
#                 weights_shape = keras_model.layers[i].get_weights()[j].shape
#                 noise = gen_normal_noise(weights_shape, scaling='')
                
#                 weight_list_dummy.append( keras_model.layers[i].get_weights()[j] + .01*noise)
#         new_model.layers[i].set_weights(weight_list_dummy) #empty if layer is empty        
#         del weight_list_dummy
#         return keras_model

# generates full keras list of weights. 
# input keras_weights: weight list returned by model.get_weights()
# output

def generate_random_weights_for_model(keras_weights):
    random_weight_list = []
    for i in range(len(keras_weights)):
        random_weight_list.append( gen_random_tensor(keras_weights[i].shape, scaling='') )
    return random_weight_list

def get_no_of_elements_to_mutate(total_weights, length_flattened_tensor, rate_mutation):
        ratio_of_the_total = length_flattened_tensor/total_weights
        return int(np.round( ratio_of_the_total*rate_mutation *length_flattened_tensor))

def generate_random_weights(keras_weights, rate_mutation, replace=1):
    total_weights = model.count_params()
#     ratio_weights_list = []
    
    new_weights = []
    for i in range(len(keras_weights)):
        #percentage of the weights to be changed at each tensor
        dummy_weights = keras_weights[i].flatten()
        length_flattened_tensor = dummy_weights.shape[0]
        no_elements_to_mutate = get_no_of_elements_to_mutate(total_weights, \
                                                             length_flattened_tensor,\
                                                             rate_mutation)
        indeces_for_random_values = random.sample(range(length_flattened_tensor),\
                                                  k=no_elements_to_mutate)
        
        if replace:
            dummy_weights[indeces_for_random_values] = \
            np.random.normal(size=(no_elements_to_mutate,))
        else:
            dummy_weights[indeces_for_random_values] = \
            1/2*(dummy_weights[indeces_for_random_values] + np.random.normal(size=(no_elements_to_mutate,)))
            
        
        dummy_weights = dummy_weights.reshape(keras_weights[i].shape)
        new_weights.append(dummy_weights)
        
    return new_weights
# def 0generate_random_weights_for_model(keras_weights):
    
def copy_model(keras_model):
    model_copy = keras.models.clone_model(keras_model, input_tensors = None)
    #     model_copy = keras.models.clone_model    
    model_copy.compile(loss=keras.losses.categorical_crossentropy,
    optimizer=keras.optimizers.Adadelta(),
    metrics=['accuracy'])
    model_copy.set_weights(keras_model.get_weights())
    return model_copy

#data is numpy array
def select_random_samples(data, labels, no_samples):
    idx = np.random.randint(data.shape[0], size=(no_samples,))
    return data[idx], labels[idx]

# def generate_
def generate_population(keras_model, no_of_individuals, mutation_rate):
    accuracies = []
    models = []
    data, labels = select_random_samples(x_train, y_train, 1000)
#     print(dir(model_copy))
    for population in range(no_of_individuals):
        model_copy = copy_model(keras_model)
#         new_model = generate_noisy_model(keras_model)
        model_copy.set_weights(generate_random_weights(model_copy.get_weights(), mutation_rate, replace=1))
        
        
        acc_dummy = model_copy.evaluate(data, labels, verbose=0)[1]
        models.append(model_copy)
        accuracies.append(acc_dummy)
#         print('generating '+ str (population) + ' individual; acc: ' + str(acc_dummy))
    return models, accuracies

#findx incedes of max values in list
def n_max_values(no_list, n):
    idx =[]
    dummy_no_list = no_list.copy()
    for i in range(n):
        idx.append(np.argmax( np.asarray(dummy_no_list) ))
        dummy_no_list.pop(idx[i])
        
    return idx

#for one dimensional number list
def softmax(no_list):
    L = -np.asarray(no_list)*10
    denominator = np.sum( np.exp(L)) 
    numerator = np.exp(L)
    evaluated = numerator/denominator
#     print('softmax' + str (evaluated))
    return evaluated

def select_fit_individuals(models, accuracies, no_to_select):
    idx = n_max_values(accuracies, no_to_select)
    selected_acc = [accuracies[i] for i in idx]
    acc_prob = softmax(selected_acc)
    selected_models = [models[i] for i in idx]
#     del models, accuracies
    
    print('highest accuracies: ' + str(selected_acc))
    return selected_models, acc_prob, selected_acc
                         
def generate_next_generation(selected_models, acc_prob, generation_population, mutation_rate):
    models_new_populations = []
    
    for i in range(len(acc_prob)):
        models_new_populations.append(int(np.round(acc_prob[i]*generation_population)))
    
#     print(models_new_populations)
#     print(acc_prob)
        
    new_models = []  
    accuracies = []
    
#     models_new_populations = np.array((generation_population, 1))
    
    
    for i in range(len(models_new_populations)):
        models, accuracy_s = generate_population(selected_models[i], models_new_populations[i], mutation_rate)
#         print(accuracy_s)
        new_models.extend(models)
        accuracies.extend(accuracy_s)
    print('new gen. accuracies: ' + str(accuracies))
    return new_models, accuracies
                         
def is_accurate_enough(accuracies, thresh):
    idx = np.argmax( np.asarray(accuracies) )
    accurate_enough = accuracies[idx] >= thresh
    return accurate_enough, idx

In [7]:
generations = 20
no_candidates_population = 20
no_to_select = 3
mutation_rate = .01

models, accuracies = generate_population(model, no_candidates_population, mutation_rate)
accurate_enough, idx = is_accurate_enough(accuracies, .9)
for i in range(generations):
    if accurate_enough:
        print('model is accurate enough.')
        break
    print('epoch ' + str(i+1))
    selected_models, acc_prob, selected_acc = select_fit_individuals(models, accuracies, no_to_select)
    del accuracies, models
    models, accuracies = generate_next_generation(selected_models, acc_prob, no_candidates_population, mutation_rate)
    accurate_enough, idx = is_accurate_enough(accuracies, .9)
    print()

epoch 1
highest accuracies: [0.178, 0.178, 0.14]
new gen. accuracies: [0.101, 0.133, 0.126, 0.128, 0.124, 0.105, 0.173, 0.115, 0.124, 0.11, 0.115, 0.159, 0.123, 0.144, 0.092, 0.138, 0.134, 0.087, 0.111, 0.136]

epoch 2
highest accuracies: [0.173, 0.115, 0.159]
new gen. accuracies: [0.14, 0.192, 0.084, 0.113, 0.17, 0.063, 0.069, 0.088, 0.143, 0.095, 0.12, 0.139, 0.097, 0.113, 0.128, 0.06, 0.121, 0.164, 0.152, 0.102]

epoch 3
highest accuracies: [0.192, 0.113, 0.06]
new gen. accuracies: [0.154, 0.176, 0.161, 0.1, 0.16, 0.154, 0.166, 0.159, 0.141, 0.062, 0.086, 0.056, 0.098, 0.11, 0.072, 0.063, 0.075, 0.08, 0.069, 0.07]

epoch 4
highest accuracies: [0.176, 0.154, 0.176]


KeyboardInterrupt: 

In [ ]:
# np.random.choice(x_train, size=5000)
# w[0]

In [ ]:
copy.copy
# x_train.shape

In [ ]:
dir(model)

In [ ]:
model.layers[1].get_weights()[0][3] == model.layers[1].get_weights()[1]

In [ ]:
model.layers[1].get_weights()[1] == model.layers[1].get_weights()[0][0][0][0] 

In [ ]:
model.metrics_names